# `arka_carbonmangels`

This notebook implements the [__Arithmetic Residuals in K-groups Analysis (ARKA)__](https://doi.org/10.1039/D4EM00173G) method in Python and then demonstrates the data on the Theraputic Data Commons [CYP2C9 Substrate CarbonMangels](https://tdcommons.ai/benchmark/admet_group/13cyp2c9s/) benchmark (via TDC and [`polaris`](https://polarishub.io/benchmarks/tdcommons/cyp2c9-substrate-carbonmangels)).

This method takes a collection of molecular descriptors and projects them into a lower dimension for subsequent regression.
The inline comments explain things in greater detail!

In [1]:
import numpy as np
from mordred import Calculator, descriptors
from rdkit.Chem import MolFromSmiles
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

## Descriptor Calculator

We'll start by defining a basic molecular descriptor calculator function.
For this demo we will use [`mordredcommunity`](https://github.com/JacksonBurns/mordred-community), a community-maintained fork of the [Mordred descriptor calculator](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-018-0258-y).

This function takes a list of SMILES strings and returns DataFrame of the 1,613 descriptors for each molecule.

In [2]:
def mordred_features(smiles_list):
    calc = Calculator(descs=descriptors, ignore_3D=True)
    mols = [MolFromSmiles(smiles) for smiles in smiles_list]
    features = calc.pandas(mols).fill_missing()
    features.replace([np.inf, -np.inf], np.nan, inplace=True)
    return features

## Benchmark Data

Based on which library you have installed, you can get to the training data using the below code - just set `DATASET_SOURCE` appropriately.

In [3]:
DATASET_SOURCE = "polaris"  # "pytdc"

In [4]:
if DATASET_SOURCE == "polaris":
    import polaris as po
    
    benchmark = po.load_benchmark("tdcommons/cyp2d6-substrate-carbonmangels")
    smiles_col = list(benchmark.input_cols)[0]
    target_col = list(benchmark.target_cols)[0]
    train, test = benchmark.get_train_test_split()
    train_df = train.as_dataframe()
    test_df = test.as_dataframe()
elif DATASET_SOURCE == "pytdc":
    from tdc.benchmark_group import admet_group

    group = admet_group(path = 'tdc_data/')
    benchmark = group.get('cyp2d6-substrate-carbonmangels')
    train_df, test_df = benchmark['train_val'], benchmark['test']
    smiles_col = "Drug"
    target_col = "Y"
else:
    raise RuntimeError("huh?")

/root/miniforge3/envs/arka/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/root/miniforge3/envs/arka/lib/python3.10/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/root/miniforge3/envs/arka/lib/python3.10/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[2025-09-30 10:30:03] INFO     The version of Polaris that was used to create the artifact (dev) is ]8;id=802862;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=36127;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/_artifact.py#96\96]8;;\
                               different from the currently installed version of Polaris (0.13.0).                 

                      INFO     To verify the checksum, we need to recompute it. This can be slow    ]8;id=524299;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/mixins/_checksum.py\_checksum.py]8;;\:]8;id=642256;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/mixins/_checksum.py#67\67]8;;\
                               for large datasets.                                                                 

/root/miniforge3/envs/arka/lib/python3.10/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[10:30:03]  Success: Fetching dataset                                                                 ]8;id=931422;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=583869;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\

                      INFO     The version of Polaris that was used to create the artifact (dev) is ]8;id=490555;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=926410;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/_artifact.py#96\96]8;;\
                               different from the currently installed version of Polaris (0.13.0).                 

                      WARNING  No checksum to verify against. Specify either the md5sum parameter   ]8;id=326400;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/mixins/_checksum.py\_checksum.py]8;;\:]8;id=755312;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/mixins/_checksum.py#60\60]8;;\
                               or store the checksum in the dataset.md5sum attribute.                              

            Success: Fetching benchmark                                                               ]8;id=145493;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=668467;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\

Now let's call that function on the training data:

In [5]:
train_df['mordred_features'] = mordred_features(train_df[smiles_col].tolist()).values.tolist()

100%|██████████| 532/532 [00:37<00:00, 14.35it/s]
/root/miniforge3/envs/arka/lib/python3.10/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
/tmp/ipykernel_1815/1441371008.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features.replace([np.inf, -np.inf], np.nan, inplace=True)


In [6]:
train_features = np.vstack(train_df['mordred_features'].values)

## ARKA Procedure

First, we rescale the training data using `MinMaxScaler` to range between 0 and 1.

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_features)
X = scaler.transform(train_features)

/root/miniforge3/envs/arka/lib/python3.10/site-packages/sklearn/utils/_array_api.py:686: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/root/miniforge3/envs/arka/lib/python3.10/site-packages/sklearn/utils/_array_api.py:706: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


We use the labels of the training data to separate these features into two groups (and pull out the targets, for later):

In [8]:
labels = train_df[target_col].values
y = train_df[target_col].values

In [9]:
group_0 = X[labels == 0]
group_1 = X[labels == 1]

Now we calculate the per-feature mean within each group, then calculate the difference between the two groups.

In [10]:
group_0_means = group_0.mean(axis=0)
group_1_means = group_1.mean(axis=0)
mean_diff = group_0_means - group_1_means
mean_diff_abs = np.abs(mean_diff)

From there, we separate the features into two "classes" based on their sign:

In [11]:
class_0_features = mean_diff > 0
class_1_features = mean_diff < 0

The paper is slightly ambiguous here - we haven't used the absolute mean differences, but it makes sense to use them in the denominator here when calculating the weights.
That's what I'll do here, using indexing into the mean arrays:

In [12]:
class_0_weights = mean_diff[class_0_features] / mean_diff_abs[class_0_features].sum()
class_1_weights = mean_diff[class_1_features] / mean_diff_abs[class_1_features].sum()

This block isn't in the original paper, but is alluded to having been done in previous work.
In short, we need to downselect the features we have available.
To do so, we'll just select those in the top 10% of weight:

In [13]:
top_0_threshold = np.percentile(np.abs(class_0_weights), 90)
top_1_threshold = np.percentile(np.abs(class_1_weights), 90)

Now again, with some indexing, we touch up the weights to reflect this:

In [14]:
# this is easy to write and fast but not space efficient
class_0_weights = np.where(np.abs(class_0_weights) >= top_0_threshold, class_0_weights, 0)
class_1_weights = np.where(np.abs(class_1_weights) >= top_1_threshold, class_1_weights, 0)
# normalize weights again
class_0_weights /= np.abs(class_0_weights).sum()
class_1_weights /= np.abs(class_1_weights).sum()

Finally, we calculate the actual ARKA features!

In [15]:
arka_0 = (X[:, class_0_features] * class_0_weights).sum(axis=1)
arka_1 = (X[:, class_1_features] * class_1_weights).sum(axis=1)

## Fitting

Now we simply fit whatever regression we want on these two features against our target vector `y`.
In this demo I'm using Random Forest, because it's robust (read: foolproof).

For TDC we need to provide 5 different sets of predictions to show the variability in the method.
This isn't required for Polaris, but we can use the models as an ensemble instead.

In [16]:
clfs = [RandomForestClassifier(random_state=42+i) for i in range(5)]
for clf in clfs:
    clf.fit(np.column_stack((arka_0, arka_1)), y)

Just as a sanity check, we'll make sure we have good accuracy on the training data:

In [17]:
clfs[0].score(np.column_stack((arka_0, arka_1)), y)

1.0

## Inference

To make predictions on the held out test data, we will go through the same procedure as above while re-using the weights we already calculated.

In [18]:
test_df['mordred_features'] = mordred_features(test_df[smiles_col].tolist()).values.tolist()
test_features = np.vstack(test_df['mordred_features'].values)
X_test = scaler.transform(test_features)
X_test_arka_0 = (X_test[:, class_0_features] * class_0_weights).sum(axis=1)
X_test_arka_1 = (X_test[:, class_1_features] * class_1_weights).sum(axis=1)

100%|██████████| 135/135 [00:11<00:00, 11.42it/s]
/root/miniforge3/envs/arka/lib/python3.10/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
/tmp/ipykernel_1815/1441371008.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features.replace([np.inf, -np.inf], np.nan, inplace=True)


In [19]:
predictions = [clf.predict(np.column_stack((X_test_arka_0, X_test_arka_1))) for clf in clfs]
probabilities = [clf.predict_proba(np.column_stack((X_test_arka_0, X_test_arka_1)))[:, 1].flatten() for clf in clfs]

## Results

Based on the backend we're using to get to the data, we can now check the results:

In [20]:
if DATASET_SOURCE == "polaris":
    probabilities = np.mean(probabilities, axis=0)
    predictions = (probabilities > 0.5).astype(int)
    results = benchmark.evaluate(predictions, probabilities).results
elif DATASET_SOURCE == "pytdc":
    results = group.evaluate_many([{benchmark['name']: p} for p in predictions])

In [30]:
to_upload = benchmark.evaluate(predictions, probabilities)
to_upload.name = "ARKA-Mordred"
to_upload.github_url = "https://github.com/JacksonBurns/arka_python/blob/fcbdbf5981280303cc6f98bf62f829289bea8bca/arka_carbonmangels.ipynb"
to_upload.description = "ARKA method using Mordred(community) molecular descriptors"
to_upload.tags = ["mordred", "mordredcommunity", "random forest", "arka"]
to_upload

{
  "benchmark_artifact_id": "tdcommons/cyp2d6-substrate-carbonmangels",
  "benchmark_name": null,
  "benchmark_owner": null,
  "results": [
    {
      "test_set": "test",
      "target_label": "Y",
      "scores": {
        "pr_auc": 0.5582998194840731
      }
    }
  ],
  "name": "ARKA-Mordred",
  "description": "ARKA method using Mordred(community) molecular descriptors",
  "tags": [
    "mordred",
    "mordredcommunity",
    "random forest",
    "arka"
  ],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "0.13.0",
  "slug": null,
  "github_url": "https://github.com/JacksonBurns/arka_python/blob/fcbdbf5981280303cc6f98bf62f829289bea8bca/arka_carbonmangels.ipynb",
  "paper_url": null,
  "contributors": [],
  "artifact_id": null,
  "urn": null,
  "version": 1
}

In [34]:
to_upload.upload_to_hub(owner="jacksonburns")

/root/miniforge3/envs/arka/lib/python3.10/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[10:43:54] Your result has been successfully uploaded to the Hub. View it here:                       ]8;id=835320;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/hub/client.py\client.py]8;;\:]8;id=58958;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/hub/client.py#527\527]8;;\
           https://polarishub.io/benchmarks/tdcommons/cyp2d6-substrate-carbonmangels/iSxqFDiodm99JkVN              
           yzUbL                                                                                                   

            Success: Uploading results                                                                ]8;id=171940;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=758096;file:///root/miniforge3/envs/arka/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\